In [1]:
import os
import pickle as pkl

In [2]:
with open('/usrvol/mappings/classes.txt', 'r') as f:
    classes = f.read().splitlines()

In [7]:
with open('/usrvol/mappings/classes.pickle', 'wb') as f:
    pkl.dump(classes, f)

In [2]:
with open('/usrvol/mappings/mapping_definitivo.pickle', 'rb') as f:
    mapping = pkl.load(f)

In [6]:
mapping['T194']

'NONE'

In [4]:
mapping['T194'] = 'NONE'

In [5]:
with open('/usrvol/mappings/mapping_definitivo.pickle', 'wb') as f:
    pkl.dump(mapping, f)

# Vocabulario y vocab_mapper obtenidos

In [1]:
import pickle as pkl

In [2]:
with open('/usrvol/mappings/categorical/vocab_mapper.pickle', 'rb') as f:
    vocab_mapper = pkl.load(f)  

In [7]:
vocab_mapper

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

In [5]:
with open('/usrvol/mappings/categorical/vocab_dict.pickle', 'rb') as f:
    vocab_dict = pkl.load(f)  

In [6]:
vocab_dict

{0: 'Enzyme, 1,4-alpha-Glucan Branching',
 1: '1 Alkyl 2 Acylphosphatidates',
 2: '1 Carboxyglutamic Acid',
 3: '1 Methyl 3 isobutylxanthine',
 4: '1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine',
 5: '1 Methyl 4 phenylpyridinium',
 6: '1 Naphthylamine',
 7: '1 Naphthylisothiocyanate',
 8: 'Angiotensin II, 1-Sarcosine-8-Isoleucine',
 9: '11 Hydroxycorticosteroids',
 10: 'synthetic 11-hydroxycorticosteroids',
 11: 'Hydrolase, 15-Ketosteryl Oleate',
 12: 'RNA, 15S',
 13: 'E2, 16,16-Dimethylprostaglandin',
 14: 'beta-Hydroxy-5 beta-Androstan-3-One, 17',
 15: 'Dehydrogenases, 17 beta-Hydroxysteroid',
 16: '17-hydroxycorticosteroid',
 17: 'Dehydrogenases, 17-Hydroxysteroid',
 18: '17-ketosteroid',
 19: 'Hydroxycorticosterone, 18',
 20: 'Hydroxydesoxycorticosterone, 18',
 21: '19 Iodocholesterol',
 22: 'Nucleotide Phosphodiesterases, 2,3-Cyclic',
 23: '2-5A Synthetase',
 24: "2' CMP",
 25: "Dipyridyl, 2,2'",
 26: 'Acid, 2,3-Diketogulonic',
 27: '2,4,5-trichlorophenoxyacetic acid',
 28: '2,4 Di

# Main para debugear

In [1]:
import pickle as pkl
from spacy_utils import *
from entities import *
from utils import *
from graph import *
from embeddings import *
import networkx as nx
from tqdm import tqdm

desambiguation_entities_number = 6000000
classes_number = 7000000
NamesToInt = {}
maximum_hopes = 3

def expanded_graph(target, entity_tree, TUI_tree, ner, le):
    if type(target) is not str and type(target) is not int:
        for parent_entity in range(len(target)):
            entity_tree.append([])
            TUI_tree.append([])
            expanded_graph(target[parent_entity], 
                           entity_tree[parent_entity], 
                           TUI_tree[parent_entity],
                           ner, le)
    else:
        doc = ner.detected_entities(str(target))
        list_linked_entities = ner.linked_entities(doc)
        entities, TUIs = le.getNextEntitiesAndTUIs(target, list_linked_entities)
        for entity in entities:
            entity_tree.append(entity)
        for TUI in TUIs:
            TUI_tree.append(TUI)
       
    return entity_tree, TUI_tree

def subgraph_generator(entities, graph, obtained_entities, TUIs, limit= 5):
    obtained_entities = list(obtained_entities)
    entity_list = []
    triple_list = []
    for entity in entities:
        if entity in graph.nodes():
            counter_in = 0
            for triple in graph.in_edges(entity, data=True):
                if triple[0] not in obtained_entities:
                    target_TUIs = graph.nodes[triple[0]]['TUIs']
                    for TUI in target_TUIs:
                        if TUI in TUIs:
                            if counter_in < limit:
                                entity_list.append(triple[0])
                                obtained_entities.append(triple[0])
                                triple_list.append((triple[0], triple[2]['rel'], triple[1]))
                                counter_in +=1
                                break
            counter_out = 0
            for triple in graph.out_edges(entity, data=True):
                if triple[1] not in obtained_entities:
                    target_TUIs = graph.nodes[triple[1]]['TUIs']
                    for TUI in target_TUIs:
                        if TUI in TUIs:
                            if counter_out < limit:
                                entity_list.append(triple[1])
                                obtained_entities.append(triple[1])
                                triple_list.append((triple[0], triple[2]['rel'], triple[1]))
                                counter_out +=1
                                break
    obtained_entities = list(set(obtained_entities))
    entity_list = list(set(entity_list))
    return entity_list, triple_list, obtained_entities


with open('/usrvol/mappings/CUIsToInt.pickle', 'rb') as f:
    CuisToInt = pkl.load(f)

with open('/usrvol/mappings/graph.pickle', 'rb') as f:
    G = pkl.load(f)    

with open('/usrvol/mappings/NamestoCUIFinal.pickle', 'rb') as f:
    NamesToCUIs = pkl.load(f)

with open('/usrvol/mappings/IntToCUIs.pickle', 'rb') as f:
    IntToCUIs = pkl.load(f)

with open('/usrvol/mappings/CUItoPreferredNamesFinal.pickle', 'rb') as f:
    CUIToNames = pkl.load(f)

with open('/usrvol/mappings/IntToNames.pickle', 'rb') as f:
    IntToNames = pkl.load(f)

with open('/usrvol/mappings/cuiToTUIs.pickle', 'rb') as f:
    CUItoTUIs = pkl.load(f)

with open('/usrvol/mappings/TUIsToInt.pickle', 'rb') as f:
    TUIToInt = pkl.load(f)

with open('/usrvol/mappings/RelationtoIdentifierTotal.pickle', 'rb') as f:
    RelationToIdentifier = pkl.load(f)

with open('/usrvol/mappings/mapping_definitivo.pickle', 'rb') as f:
    labelMapping = pkl.load(f)

with open('/usrvol/mappings/TUItoSemantic.pickle', 'rb') as f:
    TUItoText = pkl.load(f)

with open('/usrvol/mappings/classes.pickle', 'rb') as f:
    classes = pkl.load(f)

with open('/usrvol/mappings/classesToTextDict.pickle', 'rb') as f:
    classesToText = pkl.load(f)

mappers = (CuisToInt, IntToCUIs, NamesToCUIs, CUIToNames)

# función recursiva que nos expande el grafo a partir de un target devolviendo un diccionario
# con sus conexiones
ner = Ner(spacy_model='en_core_sci_lg', 
          pipeline_linker='scispacy_linker', 
          resolve_abbreviations=True, 
          ontology="umls", 
          threshold="0.85", 
          max_entities_per_mention='100')

le = Linked_Entities(resolve_abbreviations=True, ontology="umls", threshold=0.85, max_entities_per_mention=100)

text_path = '/usrvol/ISABIAL/Anotaciones_v2_ISABIAL/txt'
entries = os.listdir(text_path)
annotations_dict = {}
vocab_mapper = {}
iterator = 0
counter_mapper = 0



/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1

In [11]:
NamesToInt

{'expansion': 6000019,
 'huntingtin gene': 6000020,
 'Phosphorylation': 6000021,
 'translation initiation factor': 6000022,
 'alteration': 6000023,
 'protein synthesis': 6000024,
 'age': 6000025,
 'factor': 6000026,
 'adult': 6000027,
 'factors': 6000028,
 'maternal': 6000029,
 'diagnosis': 6000030,
 'care': 6000031,
 'primary': 6000032,
 'review': 6000033,
 'genetics': 6000034,
 'pathology': 6000035,
 'imaginal disc-derived wing expansion': 6000036,
 'genetic testing': 6000037}

In [21]:
len(NamesToInt)

0

In [2]:
IntToNames[1023783]

'imaginal disc-derived wing expansion'

In [3]:
'imaginal disc-derived wing expansion' in NamesToInt.keys()

False

In [5]:
NamesToInt['imaginal disc-derived wing expansion']

6000017

In [34]:
NamesToInt = {}
annotations_dict = {}
vocab_mapper = {}
iterator = 0
counter_mapper = 0
for class_ in classes:
    IntToNames[classes_number] = classesToText[class_]
    NamesToInt[class_] = classes_number
    vocab_mapper[classes_number] = counter_mapper
    classes_number += 1
    counter_mapper += 1

In [35]:
for file_path in tqdm(range(1000)):
    iterator += 1
    print(f"file number: {file_path}" )
    if f"text{file_path}.txt" in entries:
        file = f"text{file_path}.txt"
        with open(f"/usrvol/ISABIAL/Anotaciones_v2_ISABIAL/txt/{file}", 'r') as f:
            lines = f.readlines()
        text = lines[0]
        candidates_dict = {}
        doc = ner.detected_entities(text)
        list_linked_entities = ner.linked_entities(doc)
        candidates, candidates_TUIs = le.candidatesChecker(list_linked_entities)
        
        # Con esto evitamos repetir candidatos si aparecen varias veces en el texto
        new_candidates = []
        for candidate in candidates:
            if candidate not in new_candidates:
                new_candidates.append(candidate)
        candidates = new_candidates
        
        
        entity_list, TUI_list = le.getEntitiesAndTUIs(list_linked_entities)
        final_graphs = {}
        labels = {}

        # Creamos una lista de Nombres de entidades a desambiguar mapeandolas a un entero
        for candidate in range(len(candidates)):     
            if not candidates[candidate] in NamesToInt.keys():
                IntToNames[desambiguation_entities_number] = candidates[candidate]
                NamesToInt[candidates[candidate]] = desambiguation_entities_number
                vocab_mapper[desambiguation_entities_number] = counter_mapper
                desambiguation_entities_number += 1
                counter_mapper +=1
            
            else:
                # Si se cumple esta condición quiere decir que el término aparece tanto como candidato como en el vocabulario original
                if NamesToInt[candidates[candidate]] not in vocab_mapper.keys():
                    vocab_mapper[NamesToInt[candidates[candidate]]] = counter_mapper
                    counter_mapper +=1




        # iteramos sobre cada candidato teniendo ya el vocabulario de los candidatos   
        for candidate in range(len(candidates)):
            graph_entities = {'first_entities': [], 'last_entities': [], 'TUIs': []}
            obtained_entities = []
            TUIs = candidates_TUIs[candidate]
            forward_triples_list = []
            backward_triples_list = []
            connected_entities, _ = expanded_graph(candidates[candidate], [], [], ner, le)

            # Nos quedamos solo con las entidades que estén en el vocabulario según la base de datos
            newconnected_entities = []
            for entity in range(len(connected_entities)):
                    if connected_entities[entity] in NamesToCUIs.keys():
                        newconnected_entities.append(CuisToInt[NamesToCUIs[connected_entities[entity].lower()]])
                        
                        # Añadimos estas entidades al vocabulario
                        if not connected_entities[entity] in NamesToInt.keys():
                            IntToNames[desambiguation_entities_number] = connected_entities[entity]
                            NamesToInt[connected_entities[entity]] = desambiguation_entities_number
                            vocab_mapper[desambiguation_entities_number] = counter_mapper
                            desambiguation_entities_number += 1
                            counter_mapper +=1
            
            connected_entities = newconnected_entities
            graph_entities['first_entities'] = connected_entities
  
            # formamos el grafo desde las entidades iniciales
            for hop in range(maximum_hopes):
                connected_entities, triples, obtained_entities = subgraph_generator(connected_entities, G, obtained_entities, TUIs)
                forward_triples_list.extend(triples)
                forward_triples_list = list(set(forward_triples_list))
            Rconnected_entities, RTUIs = le.otherRelatedEntities(candidates[candidate], 
                                                        candidates, 
                                                        candidates_TUIs, 
                                                        entity_list, TUI_list)
            
            # Nos quedamos solo con las entidades que estén en el vocabulario según la base de datos
            newRconnected_entities = []
            for entity in range(len(Rconnected_entities)):
                if Rconnected_entities[entity] in NamesToCUIs.keys():
                    if not Rconnected_entities[entity] in NamesToInt.keys():
                        IntToNames[desambiguation_entities_number] = Rconnected_entities[entity]
                        NamesToInt[Rconnected_entities[entity]] = desambiguation_entities_number
                        vocab_mapper[desambiguation_entities_number] = counter_mapper
                        desambiguation_entities_number += 1
                        counter_mapper +=1

                    if NamesToCUIs[Rconnected_entities[entity].lower()] not in CuisToInt.keys():
                        CuisToInt[NamesToCUIs[Rconnected_entities[entity].lower()]] = counter_mapper
                    newRconnected_entities.append(CuisToInt[NamesToCUIs[Rconnected_entities[entity].lower()]])
            Rconnected_entities = newRconnected_entities
            graph_entities['last_entities'] = Rconnected_entities
            graph_entities['TUIs'] = RTUIs
            statements_graphs = []
           
            # añadimos los TUIs a labels para usarlos como etiquetas
            for TUI in TUIs:
                if not candidates[candidate] in labels.keys():
                    labels[candidates[candidate]] = [labelMapping[TUI]]
                else:
                    labels[candidates[candidate]].append(labelMapping[TUI])

            # Formamos todas las variantes de grafo desde las entidades finales
            for class_ in classes:
                if class_ in labels.keys():
                    graph_last_entities = []

                    # Si la entidad final de la lista de entidades contiene entre sus TUIs el TUI objetivo...
                    for entity in range(len(graph_entities['last_entities'])):
                        if TUI in graph_entities['TUIs'][entity]:
                            # Se añade a graph_last_entities para formar este grafo en particular
                            graph_last_entities.append(graph_entities['last_entities'][entity])
                    final_entities = graph_last_entities
                    # se obtienen las tripletas inversas para este TUI en particular
                    for hop in range(maximum_hopes):      
                        graph_last_entities, triples, obtained_entities = subgraph_generator(graph_last_entities, G, obtained_entities, TUIs)
                        backward_triples_list.extend(triples)
                        backward_triples_list = list(set(backward_triples_list))
                    triples_list = forward_triples_list + backward_triples_list
                    graph = nx.MultiDiGraph()
                    target = candidates[candidate]
                    initial_entities = graph_entities['first_entities']
                    final_entities = graph_entities['last_entities']
                    attrs = set()
                    for triple in triples_list:
                        head = triple[0]
                        # Añadimos todas las entidades y relaciones de las tripletas de cada grafo al vocabulario si no existen aún
                        if not IntToNames[head] in NamesToInt.keys():
                            NamesToInt[IntToNames[head]] = head
                            vocab_mapper[head] = counter_mapper
                            counter_mapper +=1
                        elif not head in vocab_mapper.keys():
                            head = NamesToInt[IntToNames[head]]

                        tail = triple[2]
                        # Idem
                        if not IntToNames[tail] in NamesToInt.keys():
                            NamesToInt[IntToNames[tail]] = tail
                            vocab_mapper[tail] = counter_mapper
                            counter_mapper +=1
                        elif not tail in vocab_mapper.keys():
                            tail = NamesToInt[IntToNames[tail]]
                        
                        graph.add_node(vocab_mapper[head], TUIs='None', node_type=2, node_score=0)
                        graph.add_node(vocab_mapper[tail], TUIs='None', node_type=2, node_score=0)
                        rel = triple[1]
                        weight = 1.
                        graph.add_edge(vocab_mapper[head], vocab_mapper[tail], rel=rel, weight=weight)
                        attrs.add((vocab_mapper[head], vocab_mapper[tail], rel))
                    # añadimos el nodo inicial y sus conexiones
                    graph.add_node(vocab_mapper[NamesToInt[target]], TUIs=TUIs, node_type=0, node_score=0)
                    for entity in initial_entities:
                        if not IntToNames[entity] in NamesToInt.keys():
                            NamesToInt[IntToNames[entity]] = entity
                            vocab_mapper[entity] = counter_mapper
                            counter_mapper +=1
                        elif not entity in vocab_mapper.keys():
                            entity = NamesToInt[IntToNames[entity]]
                        if not entity in graph.nodes:
                            try:
                                T = CUItoTUIs[IntToCUIs[entity]]
                            except:
                                T = 'NOT FOUND'
                        graph.add_node(vocab_mapper[entity], TUIs=[T], node_type=2, node_score=0)
                        graph.add_edge(vocab_mapper[NamesToInt[target]], vocab_mapper[entity], rel=RelationToIdentifier['meaning_of_concept'], weight=1.)
                    # añadimos el nodo final y sus conexiones
                    graph.add_node(vocab_mapper[NamesToInt[class_]], TUIs=['None'], node_type=1, node_score=0)
                    for entity in final_entities:
                        if not IntToNames[entity] in NamesToInt.keys():
                            NamesToInt[IntToNames[entity]] = entity
                            vocab_mapper[entity] = counter_mapper
                            counter_mapper +=1
                        elif not entity in vocab_mapper.keys():
                            entity = NamesToInt[IntToNames[entity]]
                        if not entity in graph.nodes:
                            try:
                                T = CUItoTUIs[IntToCUIs[entity]]
                            except:
                                T = 'NOT FOUND'
                        graph.add_node(vocab_mapper[entity], TUIs=['None'], node_type=2, node_score=0)
                        graph.add_edge(vocab_mapper[entity], vocab_mapper[NamesToInt[class_]], rel=RelationToIdentifier['belongs_to'], weight=1.)
                    statements_graphs.append(graph)
                    backward_triples_list = []
                
                else:
                    triples_list = forward_triples_list
                    graph = nx.MultiDiGraph()
                    target = candidates[candidate]
                    initial_entities = graph_entities['first_entities']
                    attrs = set()
                    for triple in triples_list:
                        
                        
                        head = triple[0]
                        # Añadimos todas las entidades y relaciones de las tripletas de cada grafo al vocabulario si no existen aún
                        if not IntToNames[head] in NamesToInt.keys():
                            NamesToInt[IntToNames[head]] = head
                            vocab_mapper[head] = counter_mapper
                            counter_mapper +=1
                        # Esto quiere decir que el término aparece tanto como candidato como en el vocabulario original, teniendo dos números diferentes que vamos a mapear a uno solo
                        elif not head in vocab_mapper.keys():
                            head = NamesToInt[IntToNames[head]]
                        
                        
                        tail = triple[2]
                        # Idem
                        if not IntToNames[tail] in NamesToInt.keys():
                            NamesToInt[IntToNames[tail]] = tail
                            vocab_mapper[tail] = counter_mapper
                            counter_mapper +=1
                        # Esto quiere decir que el término aparece tanto como candidato como en el vocabulario original, teniendo dos números diferentes que vamos a mapear a uno solo
                        elif not tail in vocab_mapper.keys():
                            tail = NamesToInt[IntToNames[tail]]


                        graph.add_node(vocab_mapper[head], TUIs='none', node_type=2, node_score=0)
                        graph.add_node(vocab_mapper[tail], TUIs='none', node_type=2, node_score=0)
                        rel = triple[1]
                        weight = 1.
                        graph.add_edge(vocab_mapper[head], vocab_mapper[tail], rel=rel, weight=weight)
                        attrs.add((vocab_mapper[head], vocab_mapper[tail], rel))
                    # añadimos el nodo inicial y sus conexiones
                    graph.add_node(vocab_mapper[NamesToInt[target]], TUIs=TUIs, node_type=0, node_score=0)
                    for entity in initial_entities:
                        if not IntToNames[entity] in NamesToInt.keys():
                            NamesToInt[IntToNames[entity]] = entity
                            vocab_mapper[entity] = counter_mapper
                            counter_mapper +=1

                        # Esto quiere decir que el término aparece tanto como candidato como en el vocabulario original, teniendo dos números diferentes que vamos a mapear a uno solo
                        elif not entity in vocab_mapper.keys():
                            entity = NamesToInt[IntToNames[entity]]
                        if not entity in graph.nodes:
                            try:
                                T = CUItoTUIs[IntToCUIs[entity]]
                            except:
                                T = 'NOT FOUND'     
                        graph.add_node(vocab_mapper[entity], TUIs=[T], node_type=2, node_score=0)
                        graph.add_edge(vocab_mapper[NamesToInt[target]], vocab_mapper[entity], rel=RelationToIdentifier['meaning_of_concept'], weight=1.)
                    # añadimos el nodo final y sin conexiones porque no es su clase
                    graph.add_node(vocab_mapper[NamesToInt[class_]], TUIs=['None'], node_type=1, node_score=0)
                    statements_graphs.append(graph)           
            final_graphs[candidates[candidate]] = statements_graphs


        # Debemos eliminar todas las opciones que aparezcan con un NONE
        candidates_list = []
        for key in labels.keys():
            new_labels = []
            new_graphs = []
            for i in range(len(labels[key])):
                if labels[key][i] != 'NONE':
                    new_labels.append(labels[key][i])
            new_labels = list(set(new_labels))
            if len(new_labels) < 1:
                candidates_list.append(key)
            labels[key] = new_labels
            final_graphs[key] = new_graphs

        for key in candidates_list:
            del labels[key]

        with open(f"/usrvol/data/categorical/labels{iterator}.pickle", 'wb') as f:
            pkl.dump(labels, f)
            
        with open(f"/usrvol/data/categorical/final_graphs{iterator}.pickle", 'wb') as f:
            pkl.dump(final_graphs, f)


# En lugar de usar vocab_dict mapeamos con vocab_mapper usando intToNames
reverse_vocab_mapper = {v: k for k, v in vocab_mapper.items()}
with open("/usrvol/mappings/categorical/vocab.txt", 'w') as f:
    for i in range(len(reverse_vocab_mapper)):
        value = IntToNames[reverse_vocab_mapper[i]]
        print(value, file=f)

with open('/usrvol/mappings/categorical/vocab_mapper.pickle', 'wb') as f:
            pkl.dump(vocab_mapper, f)
print("")

  0%|          | 1/1000 [00:00<02:40,  6.24it/s]

file number: 0
file number: 1


  0%|          | 3/1000 [00:00<02:15,  7.34it/s]

file number: 2
file number: 3


  0%|          | 5/1000 [00:00<02:44,  6.04it/s]

...

  6%|▋         | 65/1000 [00:11<04:13,  3.69it/s]

file number: 65


  7%|▋         | 67/1000 [00:11<03:19,  4.68it/s]

file number: 66
file number: 67


  7%|▋         | 68/1000 [00:11<03:17,  4.72it/s]

In [31]:
vocab_mapper[547367]

KeyError: 547367

In [32]:
IntToNames[547367]

'landscape'

In [16]:
NamesToInt['actin phosphorylation']

6000057

In [33]:
NamesToInt[IntToNames[547367]]

6003346

# Annotation dict

In [1]:
import pickle as pkl

In [14]:
with open('/usrvol/data/categorical/final_labels.pickle', 'rb') as f:
    final_labels = pkl.load(f)

In [15]:
final_labels

{1: {'huntingtin gene': ['GeneOrGenome'],
  'Phosphorylation': ['CellFunction'],
  'translation initiation factor': ['AminoAcidPeptideOrProtein',
   'BiologicallyActiveSubstance'],
  'protein synthesis': ['CellFunction'],
  'adult': ['PopulationGroup'],
  'maternal': ['PopulationGroup'],
  'diagnosis': ['HealthCareActivity'],
  'genetics': ['BiomedicalOccupationOrDiscipline'],
  'pathology': ['PathologicFunction', 'BiomedicalOccupationOrDiscipline']},
 2: {'muscle stiffness': ['SignOrSymptom'],
  'symptoms': ['SignOrSymptom'],
  'dopamine': ['BiologicallyActiveSubstance']},
 3: {'Mitochondria': ['CellComponent'],
  'health': ['HealthCareActivity'],
  'metabolic process': ['CellFunction'],
  'mitochondria': ['CellComponent']},
 5: {'symptom': ['SignOrSymptom'],
  'HD': ['DiseaseOrSyndrome'],
  'pathophysiology': ['PathologicFunction']},
 7: {"Alzheimer's": ['DiseaseOrSyndrome'],
  'Huntington disease': ['DiseaseOrSyndrome'],
  'quality of life': ['Finding'],
  'treatment': ['HealthCareA

In [11]:
with open('/usrvol/data/categorical/graphs/final_graphs1.pickle', 'rb') as f:
    final_graphs = pkl.load(f)

In [13]:
final_graphs.keys()

dict_keys(['expansion', 'huntingtin gene', 'Phosphorylation', 'translation initiation factor', 'alteration', 'protein synthesis', 'age', 'factor', 'adult', 'factors', 'maternal', 'diagnosis', 'care', 'primary', 'review', 'genetics', 'pathology'])

# Comprobamos el tamaño de los grafos obtenidos

In [1]:
import pickle as pkl

In [3]:
max_len = 0
for iterator in range(1, 1001):
    try:
        with open(f'/usrvol/data/categorical/graphs/final_graphs{iterator}.pickle', 'rb') as f:
            final_graphs = pkl.load(f)
    except:
        continue
    for term in final_graphs.keys():
        for graph in final_graphs[term]:
            if len(graph.nodes) > max_len:
                max_len = len(graph.nodes)

print(max_len)

282


# Vemos los datos de entrada al modelo

In [2]:
import pickle as pkl

In [9]:
with open('/usrvol/data/categorical/graph_statements_tensors/graph_statementsdev.pickle', 'rb') as f:
    graphs = pkl.load(f)

In [28]:
graphs[0].size()

torch.Size([8160, 1, 300])

# Modidicamos el generador de grafos hasta obtener tamaños adecuados

In [2]:
import pickle as pkl
from spacy_utils import *
from entities import *
from utils import *
from graph import *
from embeddings import *
import networkx as nx
from tqdm import tqdm


/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def expanded_graph(target, entity_tree, TUI_tree, ner, le):
    if type(target) is not str and type(target) is not int:
        for parent_entity in range(len(target)):
            entity_tree.append([])
            TUI_tree.append([])
            expanded_graph(target[parent_entity], 
                           entity_tree[parent_entity], 
                           TUI_tree[parent_entity],
                           ner, le)
    else:
        doc = ner.detected_entities(str(target))
        list_linked_entities = ner.linked_entities(doc)
        entities, TUIs = le.getNextEntitiesAndTUIs(target, list_linked_entities)
        for entity in entities:
            entity_tree.append(entity)
        for TUI in TUIs:
            TUI_tree.append(TUI)
       
    return entity_tree, TUI_tree

def subgraph_generator(entities, graph, obtained_entities, TUIs, limit= 200, total_limit = 1000):
    obtained_entities = list(obtained_entities)
    entity_list = []
    triple_list = []
    for entity in entities:
        if entity in graph.nodes():
            counter_in = 0
            for triple in graph.in_edges(entity, data=True):
                if triple[0] not in obtained_entities:
                    if counter_in < limit:
                        entity_list.append(triple[0])
                        obtained_entities.append(triple[0])
                        triple_list.append((triple[0], triple[2]['rel'], triple[1]))
                        counter_in +=1
                        break
            counter_out = 0
            for triple in graph.out_edges(entity, data=True):
                if triple[1] not in obtained_entities:
                    if counter_out < limit:
                        entity_list.append(triple[1])
                        obtained_entities.append(triple[1])
                        triple_list.append((triple[0], triple[2]['rel'], triple[1]))
                        counter_out +=1
                        break
    obtained_entities = list(set(obtained_entities))
    entity_list = list(set(entity_list))
    if len(obtained_entities) > total_limit:
        obtained_entities = obtained_entities[:total_limit]
        entity_list = entity_list[:total_limit]
        for triple in triple_list:
            if triple[0] not in obtained_entities:
                triple_list.remove(triple)
                continue
            if triple[1] not in obtained_entities:
                triple_list.remove(triple)
    return entity_list, triple_list, obtained_entities

In [4]:

with open('/usrvol/mappings/CUIsToInt.pickle', 'rb') as f:
    CuisToInt = pkl.load(f)

with open('/usrvol/mappings/graph.pickle', 'rb') as f:
    G = pkl.load(f)    

with open('/usrvol/mappings/NamestoCUIFinal.pickle', 'rb') as f:
    NamesToCUIs = pkl.load(f)

with open('/usrvol/mappings/IntToCUIs.pickle', 'rb') as f:
    IntToCUIs = pkl.load(f)

with open('/usrvol/mappings/CUItoPreferredNamesFinal.pickle', 'rb') as f:
    CUIToNames = pkl.load(f)

with open('/usrvol/mappings/IntToNames.pickle', 'rb') as f:
    IntToNames = pkl.load(f)

with open('/usrvol/mappings/cuiToTUIs.pickle', 'rb') as f:
    CUItoTUIs = pkl.load(f)

with open('/usrvol/mappings/TUIsToInt.pickle', 'rb') as f:
    TUIToInt = pkl.load(f)

with open('/usrvol/mappings/RelationtoIdentifierTotal.pickle', 'rb') as f:
    RelationToIdentifier = pkl.load(f)

with open('/usrvol/mappings/mapping_definitivo.pickle', 'rb') as f:
    labelMapping = pkl.load(f)

with open('/usrvol/mappings/TUItoSemantic.pickle', 'rb') as f:
    TUItoText = pkl.load(f)

with open('/usrvol/mappings/classes.pickle', 'rb') as f:
    classes = pkl.load(f)

with open('/usrvol/mappings/classesToTextDict.pickle', 'rb') as f:
    classesToText = pkl.load(f)

mappers = (CuisToInt, IntToCUIs, NamesToCUIs, CUIToNames)

In [5]:
# función recursiva que nos expande el grafo a partir de un target devolviendo un diccionario
# con sus conexiones
ner = Ner(spacy_model='en_core_sci_lg', 
          pipeline_linker='scispacy_linker', 
          resolve_abbreviations=True, 
          ontology="umls", 
          threshold="0.85", 
          max_entities_per_mention='100')

le = Linked_Entities(resolve_abbreviations=True, ontology="umls", threshold=0.85, max_entities_per_mention=100)

/usr/local/lib/python3.10/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
text_path = '/usrvol/ISABIAL/Anotaciones_v2_ISABIAL/txt'
entries = os.listdir(text_path)
annotations_dict = {}
vocab_mapper = {}
iterator = 0
counter_mapper = 0
desambiguation_entities_number = 6000000
classes_number = 7000000
NamesToInt = {}
maximum_hopes = 3

In [ ]:
# Este solo lo ejecutamos una vez
for class_ in classes:
    IntToNames[classes_number] = classesToText[class_]
    NamesToInt[class_] = classes_number
    vocab_mapper[classes_number] = counter_mapper
    classes_number += 1
    counter_mapper += 1

In [1]:
for file_path in tqdm(range(1000)):
    iterator += 1
    print(f"file number: {file_path}" )
    if f"text{file_path}.txt" in entries:
        file = f"text{file_path}.txt"
        with open(f"/usrvol/ISABIAL/Anotaciones_v2_ISABIAL/txt/{file}", 'r') as f:
            lines = f.readlines()
        text = lines[0]
        candidates_dict = {}
        doc = ner.detected_entities(text)
        list_linked_entities = ner.linked_entities(doc)
        candidates, candidates_TUIs = le.candidatesChecker(list_linked_entities)
        
        # Con esto evitamos repetir candidatos si aparecen varias veces en el texto
        new_candidates = []
        for candidate in candidates:
            if candidate not in new_candidates:
                new_candidates.append(candidate)
        candidates = new_candidates
        
        
        entity_list, TUI_list = le.getEntitiesAndTUIs(list_linked_entities)
        final_graphs = {}
        labels = {}

        # Creamos una lista de Nombres de entidades a desambiguar mapeandolas a un entero
        for candidate in range(len(candidates)):     
            if not candidates[candidate] in NamesToInt.keys():
                IntToNames[desambiguation_entities_number] = candidates[candidate]
                NamesToInt[candidates[candidate]] = desambiguation_entities_number
                vocab_mapper[desambiguation_entities_number] = counter_mapper
                desambiguation_entities_number += 1
                counter_mapper +=1
            
            else:
                # Si se cumple esta condición quiere decir que el término aparece tanto como candidato como en el vocabulario original
                if NamesToInt[candidates[candidate]] not in vocab_mapper.keys():
                    vocab_mapper[NamesToInt[candidates[candidate]]] = counter_mapper
                    counter_mapper +=1

        # iteramos sobre cada candidato teniendo ya el vocabulario de los candidatos   
        for candidate in range(len(candidates)):
            graph_entities = {'first_entities': [], 'last_entities': [], 'TUIs': []}
            obtained_entities = []
            TUIs = candidates_TUIs[candidate]
            forward_triples_list = []
            backward_triples_list = []
            connected_entities, _ = expanded_graph(candidates[candidate], [], [], ner, le)

            # Nos quedamos solo con las entidades que estén en el vocabulario según la base de datos
            newconnected_entities = []
            for entity in range(len(connected_entities)):
                    if connected_entities[entity] in NamesToCUIs.keys():
                        newconnected_entities.append(CuisToInt[NamesToCUIs[connected_entities[entity].lower()]])
                        
                        # Añadimos estas entidades al vocabulario
                        if not connected_entities[entity] in NamesToInt.keys():
                            IntToNames[desambiguation_entities_number] = connected_entities[entity]
                            NamesToInt[connected_entities[entity]] = desambiguation_entities_number
                            vocab_mapper[desambiguation_entities_number] = counter_mapper
                            desambiguation_entities_number += 1
                            counter_mapper +=1
            
            connected_entities = newconnected_entities
            graph_entities['first_entities'] = connected_entities
  
            # formamos el grafo desde las entidades iniciales
            for hop in range(maximum_hopes):
                connected_entities, triples, obtained_entities = subgraph_generator(connected_entities, G, obtained_entities, TUIs)
                forward_triples_list.extend(triples)
                forward_triples_list = list(set(forward_triples_list))
            Rconnected_entities, RTUIs = le.otherRelatedEntities(candidates[candidate], 
                                                        candidates, 
                                                        candidates_TUIs, 
                                                        entity_list, TUI_list)
            
            # Nos quedamos solo con las entidades que estén en el vocabulario según la base de datos
            newRconnected_entities = []
            for entity in range(len(Rconnected_entities)):
                if Rconnected_entities[entity] in NamesToCUIs.keys():
                    if not Rconnected_entities[entity] in NamesToInt.keys():
                        IntToNames[desambiguation_entities_number] = Rconnected_entities[entity]
                        NamesToInt[Rconnected_entities[entity]] = desambiguation_entities_number
                        vocab_mapper[desambiguation_entities_number] = counter_mapper
                        desambiguation_entities_number += 1
                        counter_mapper +=1

                    if NamesToCUIs[Rconnected_entities[entity].lower()] not in CuisToInt.keys():
                        CuisToInt[NamesToCUIs[Rconnected_entities[entity].lower()]] = counter_mapper
                    newRconnected_entities.append(CuisToInt[NamesToCUIs[Rconnected_entities[entity].lower()]])
            Rconnected_entities = newRconnected_entities
            graph_entities['last_entities'] = Rconnected_entities
            graph_entities['TUIs'] = RTUIs
            statements_graphs = []
           
            # añadimos los TUIs a labels para usarlos como etiquetas
            for TUI in TUIs:
                if not candidates[candidate] in labels.keys():
                    labels[candidates[candidate]] = [labelMapping[TUI]]
                else:
                    labels[candidates[candidate]].append(labelMapping[TUI])

            # Formamos todas las variantes de grafo desde las entidades finales
            for class_ in classes:
                if class_ in labels.keys():
                    graph_last_entities = []

                    # Si la entidad final de la lista de entidades contiene entre sus TUIs el TUI objetivo...
                    for entity in range(len(graph_entities['last_entities'])):
                        if TUI in graph_entities['TUIs'][entity]:
                            # Se añade a graph_last_entities para formar este grafo en particular
                            graph_last_entities.append(graph_entities['last_entities'][entity])
                    if len(graph_last_entities) < 30:
                        final_entities = graph_last_entities
                    else:
                        final_entities = graph_last_entities[:30]

                    print("final_entities added: ", len(final_entities))
                    # se obtienen las tripletas inversas para este TUI en particular
                    for hop in range(maximum_hopes):      
                        graph_last_entities, triples, obtained_entities = subgraph_generator(graph_last_entities, G, obtained_entities, TUIs)
                        backward_triples_list.extend(triples)
                        backward_triples_list = list(set(backward_triples_list))
                    triples_list = forward_triples_list + backward_triples_list
                    graph = nx.MultiDiGraph()
                    target = candidates[candidate]
                    initial_entities = graph_entities['first_entities']
                    attrs = set()
                    for triple in triples_list:
                        head = triple[0]
                        # Añadimos todas las entidades y relaciones de las tripletas de cada grafo al vocabulario si no existen aún
                        if not IntToNames[head] in NamesToInt.keys():
                            NamesToInt[IntToNames[head]] = head
                            vocab_mapper[head] = counter_mapper
                            counter_mapper +=1
                        elif not head in vocab_mapper.keys():
                            head = NamesToInt[IntToNames[head]]
                        tail = triple[2]
                        # Idem
                        if not IntToNames[tail] in NamesToInt.keys():
                            NamesToInt[IntToNames[tail]] = tail
                            vocab_mapper[tail] = counter_mapper
                            counter_mapper +=1
                        elif not tail in vocab_mapper.keys():
                            tail = NamesToInt[IntToNames[tail]]   
                        graph.add_node(vocab_mapper[head], TUIs='None', node_type=2, node_score=0)
                        graph.add_node(vocab_mapper[tail], TUIs='None', node_type=2, node_score=0)
                        rel = triple[1]
                        weight = 1.
                        graph.add_edge(vocab_mapper[head], vocab_mapper[tail], rel=rel, weight=weight)
                        attrs.add((vocab_mapper[head], vocab_mapper[tail], rel))
                    # añadimos el nodo inicial y sus conexiones
                    graph.add_node(vocab_mapper[NamesToInt[target]], TUIs=TUIs, node_type=0, node_score=0)
                    for entity in initial_entities:
                        if not IntToNames[entity] in NamesToInt.keys():
                            NamesToInt[IntToNames[entity]] = entity
                            vocab_mapper[entity] = counter_mapper
                            counter_mapper +=1
                        elif not entity in vocab_mapper.keys():
                            entity = NamesToInt[IntToNames[entity]]
                        if not entity in graph.nodes:
                            try:
                                T = CUItoTUIs[IntToCUIs[entity]]
                            except:
                                T = 'NOT FOUND'
                        graph.add_node(vocab_mapper[entity], TUIs=[T], node_type=2, node_score=0)
                        graph.add_edge(vocab_mapper[NamesToInt[target]], vocab_mapper[entity], rel=RelationToIdentifier['meaning_of_concept'], weight=1.)
                        
                    
                    # añadimos el nodo final y sus conexiones
                    graph.add_node(vocab_mapper[NamesToInt[class_]], TUIs=['None'], node_type=1, node_score=0)
                    for entity in final_entities:
                        if not IntToNames[entity] in NamesToInt.keys():
                            NamesToInt[IntToNames[entity]] = entity
                            vocab_mapper[entity] = counter_mapper
                            counter_mapper +=1
                        elif not entity in vocab_mapper.keys():
                            entity = NamesToInt[IntToNames[entity]]
                        if not entity in graph.nodes:
                            try:
                                T = CUItoTUIs[IntToCUIs[entity]]
                            except:
                                T = 'NOT FOUND'
                        graph.add_node(vocab_mapper[entity], TUIs=['None'], node_type=2, node_score=0)
                        graph.add_edge(vocab_mapper[entity], vocab_mapper[NamesToInt[class_]], rel=RelationToIdentifier['belongs_to'], weight=1.)
                    statements_graphs.append(graph)
                    print(graph.number_of_nodes())
                    backward_triples_list = []
                else:
                    triples_list = forward_triples_list
                    graph = nx.MultiDiGraph()
                    target = candidates[candidate]
                    initial_entities = graph_entities['first_entities']
                    print("initial_entities added: ", len(initial_entities))
                    attrs = set()
                    for triple in triples_list:             
                        head = triple[0]
                        # Añadimos todas las entidades y relaciones de las tripletas de cada grafo al vocabulario si no existen aún
                        if not IntToNames[head] in NamesToInt.keys():
                            NamesToInt[IntToNames[head]] = head
                            vocab_mapper[head] = counter_mapper
                            counter_mapper +=1
                        # Esto quiere decir que el término aparece tanto como candidato como en el vocabulario original, teniendo dos números diferentes que vamos a mapear a uno solo
                        elif not head in vocab_mapper.keys():
                            head = NamesToInt[IntToNames[head]]       
                        tail = triple[2]
                        # Idem
                        if not IntToNames[tail] in NamesToInt.keys():
                            NamesToInt[IntToNames[tail]] = tail
                            vocab_mapper[tail] = counter_mapper
                            counter_mapper +=1
                        # Esto quiere decir que el término aparece tanto como candidato como en el vocabulario original, teniendo dos números diferentes que vamos a mapear a uno solo
                        elif not tail in vocab_mapper.keys():
                            tail = NamesToInt[IntToNames[tail]]

                        graph.add_node(vocab_mapper[head], TUIs='none', node_type=2, node_score=0)
                        graph.add_node(vocab_mapper[tail], TUIs='none', node_type=2, node_score=0)
                        rel = triple[1]
                        weight = 1.
                        graph.add_edge(vocab_mapper[head], vocab_mapper[tail], rel=rel, weight=weight)
                        attrs.add((vocab_mapper[head], vocab_mapper[tail], rel))
                    # añadimos el nodo inicial y sus conexiones
                    graph.add_node(vocab_mapper[NamesToInt[target]], TUIs=TUIs, node_type=0, node_score=0)
                    for entity in initial_entities:
                        if not IntToNames[entity] in NamesToInt.keys():
                            NamesToInt[IntToNames[entity]] = entity
                            vocab_mapper[entity] = counter_mapper
                            counter_mapper +=1
                        # Esto quiere decir que el término aparece tanto como candidato como en el vocabulario original, teniendo dos números diferentes que vamos a mapear a uno solo
                        elif not entity in vocab_mapper.keys():
                            entity = NamesToInt[IntToNames[entity]]

                        if not entity in graph.nodes:
                            try:
                                T = CUItoTUIs[IntToCUIs[entity]]
                            except:
                                T = 'NOT FOUND'     
                        graph.add_node(vocab_mapper[entity], TUIs=[T], node_type=2, node_score=0)
                        graph.add_edge(vocab_mapper[NamesToInt[target]], vocab_mapper[entity], rel=RelationToIdentifier['meaning_of_concept'], weight=1.)
                    # añadimos el nodo final y sin conexiones porque no es su clase
                    graph.add_node(vocab_mapper[NamesToInt[class_]], TUIs=['None'], node_type=1, node_score=0)
                    statements_graphs.append(graph)
                    print(graph.number_of_nodes())           
            final_graphs[candidates[candidate]] = statements_graphs

NameError: name 'tqdm' is not defined